In [1]:
# The following Spark basics are performed on people.json
!pip install pyspark


     |████████████████████████████████| 204.7MB 64kB/s 
     |████████████████████████████████| 204kB 45.2MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.0-py2.py3-none-any.whl size=205044182 sha256=5496dfb26589bcadb9d72b843d72122e817d7b821f6ae1054abad6ee194d07da
  Stored in directory: /root/.cache/pip/wheels/57/27/4d/ddacf7143f8d5b76c45c61ee2e43d9f8492fc5a8e78ebd7d37
Successfully built pyspark


In [5]:
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder.appName('Missing_data').getOrCreate()

In [8]:
df = spark.read.csv('/content/ContainsNull.csv', inferSchema = True, header = True)

In [9]:
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [10]:
df.na.drop().show()
# The above command drops missing data

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [11]:
df.na.drop(thresh = 2).show()
#Keeps data that has atleast 2 non null values


+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [12]:
df.na.drop(how = 'all').show()
#Drops data that has all null values
df.na.drop(how = 'any').show()
#Drops data that has any null values
df.na.drop(subset = ['sales']).show()
#Drops data that has null values in sales

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [13]:
#Replacing null values with other known values
df_1 = df.na.fill('Not Defined', subset = ['Name'])
df_1.na.fill(0, subset = ['Sales']).show()

+----+-----------+-----+
|  Id|       Name|Sales|
+----+-----------+-----+
|emp1|       John|  0.0|
|emp2|Not Defined|  0.0|
|emp3|Not Defined|345.0|
|emp4|      Cindy|456.0|
+----+-----------+-----+



In [14]:
from pyspark.sql.functions import mean

In [15]:
mean_val = df.select(mean(df['Sales'])).collect()
mean_sales = mean_val[0][0]
df.na.fill(mean_sales, subset = ['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+

